In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
usedcars = pd.read_csv('https://raw.githubusercontent.com/RoobyDoobyDoo/CS5530-Assign2/refs/heads/main/Used_Cars/train.csv')
display(usedcars)

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.4 kmpl,1248 CC,74 bhp,5.0,7.88 Lakh,4.75
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.4 kmpl,1120 CC,71 bhp,5.0,NaN,4.00
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.0 kmpl,2498 CC,112 bhp,8.0,NaN,2.90
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.9 kmpl,998 CC,67.1 bhp,5.0,NaN,2.65


In [ ]:
# Check
usedcars.info()
usedcars.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5847 non-null   int64  
 1   Name               5847 non-null   object 
 2   Location           5847 non-null   object 
 3   Year               5847 non-null   int64  
 4   Kilometers_Driven  5847 non-null   int64  
 5   Fuel_Type          5847 non-null   object 
 6   Transmission       5847 non-null   object 
 7   Owner_Type         5847 non-null   object 
 8   Mileage            5845 non-null   object 
 9   Engine             5811 non-null   object 
 10  Power              5811 non-null   object 
 11  Seats              5809 non-null   float64
 12  New_Price          815 non-null    object 
 13  Price              5847 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 639.6+ KB


,Unnamed: 0,Year,Kilometers_Driven,Seats,Price
count,5847.000000,5847.000000,5.847000e+03,5809.000000,5847.000000
mean,3013.181461,2013.448435,5.841013e+04,5.286452,9.653742
std,1736.398890,3.194949,9.237971e+04,0.806668,11.275966
min,1.000000,1998.000000,1.710000e+02,2.000000,0.440000
25%,1509.500000,2012.000000,3.346750e+04,5.000000,3.550000
50%,3015.000000,2014.000000,5.257600e+04,5.000000,5.750000
75%,4517.500000,2016.000000,7.249050e+04,5.000000,10.250000
max,6018.000000,2019.000000,6.500000e+06,10.000000,160.000000


This data is really messy. It may need a lot of cleanup.

In [ ]:
# Look for missing values
usedcars.isnull().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36


In [ ]:
# Check the distribution for all the feature data
for col in usedcars.columns:
    fig = px.histogram(usedcars, x=col)
    fig.show()

This needs the cleanup before we do any proper imputing.

B) Remove the units from some of the attributes and only keep the numerical values.

In [ ]:
# Lets' start cleaning this dataset
# We don't need unnamed
usedcars = usedcars.drop(columns=['Unnamed: 0'])

# Engine
usedcars['Engine (CC)'] = pd.to_numeric(usedcars['Engine'].str.replace(' CC', ''))
usedcars = usedcars.drop(columns=['Engine'])

# Power
usedcars['Power (bhp)'] = pd.to_numeric(usedcars['Power'].str.replace(' bhp', ''))
usedcars = usedcars.drop(columns=['Power'])

In [ ]:
# Mileage
# Some of these had km/kg, so I'll assume it's correct
# And set up a generalized conversion for it using fuel density
density = {'Petrol': 0.74, 'Diesel': 0.84}

# Get numerical data from rows
num = usedcars['Mileage'].str.extract(r'([\d\.]+)').astype(float)

# Get units
unit = usedcars['Mileage'].str.extract(r'(km/kg|kmpl)')   # Determine units
fuel_density = usedcars['Fuel_Type'].map(density)         # Map densities

# Make the coloumn with unified units
usedcars['Mileage (kmpl)'] = np.where(
    unit[0] == 'km/kg',
    # IDK why, but we have to divide by the density
    # Unit analysis tell me we should be multiplying
    # But whgen when I look up the actual kmpl of these cars,
    num[0] / fuel_density,    # it's closer to this
    num[0])

usedcars = usedcars.drop(columns=['Mileage'])   # Get rid of old col

In [ ]:
# New_Price
# Similar to what we did with mileage before
price = usedcars['New_Price'].str.extract(r'([\d\.]+)').astype(float)

currency = usedcars['New_Price'].str.extract(r'(Cr|Lakh)')

usedcars['New_Price (Lakh)'] = np.where(
    currency[0] == 'Cr',
    price[0] * 100,
    price[0])

usedcars = usedcars.drop(columns=['New_Price'])

In [ ]:
# Check ourselves
usedcars.info()

# # Let's see the distribution now
# for col in usedcars.columns:
#     fig = px.histogram(usedcars, x=col, nbins=10)
#     fig.show()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               5847 non-null   object 
 1   Location           5847 non-null   object 
 2   Year               5847 non-null   int64  
 3   Kilometers_Driven  5847 non-null   int64  
 4   Fuel_Type          5847 non-null   object 
 5   Transmission       5847 non-null   object 
 6   Owner_Type         5847 non-null   object 
 7   Seats              5809 non-null   float64
 8   Price              5847 non-null   float64
 9   Engine (CC)        5811 non-null   float64
 10  Power (bhp)        5811 non-null   float64
 11  Mileage (kmpl)     5845 non-null   float64
 12  New_Price (Lakh)   815 non-null    float64
dtypes: float64(6), int64(2), object(5)
memory usage: 594.0+ KB


C) Change the categorical variables into numerical one-hot encoded value.

In [ ]:
# One hot encode fuel type and transmission
usedcars = pd.get_dummies(usedcars, columns=['Fuel_Type', 'Transmission'], dtype=int)
# usedcars.info()

A) Look for the missing values in all columns and either impute them or drop them
- Too many values are missing in 'New_Price' to be useful, drop that whole column
- Imputing the rest should be fine


In [ ]:
# Drop New_Price becuase it has too many missing values anyway
usedcars = usedcars.drop(columns=['New_Price (Lakh)'])

# Set up imputer
imputer = IterativeImputer()

# Impute mileage from engine, year
imputed = imputer.fit_transform(usedcars[['Mileage (kmpl)',
                                          'Engine (CC)', 'Year']])
usedcars['Mileage (kmpl)'] = imputed[:, 0]

# Impute engine, would impute from name but it's too much to one-hot encode
imputed = imputer.fit_transform(usedcars[['Engine (CC)']])
usedcars['Engine (CC)'] = imputed[:, 0]

# Impute power from engine and year
imputed = imputer.fit_transform(usedcars[['Power (bhp)', 'Engine (CC)','Year']])
usedcars['Power (bhp)'] = imputed[:, 0]

# Impute seat from name as well
usedcars['Seats'] = imputer.fit_transform(usedcars[['Seats']])

# Also, get that car with 6.5 million km driven, impute it by mileage and year
maxrealkm = 1000000
usedcars['Kilometers_Driven'] = np.where(
    usedcars['Kilometers_Driven'] > maxrealkm,
    np.nan, usedcars['Kilometers_Driven'])

imputed = imputer.fit_transform(usedcars[['Kilometers_Driven', 'Year']])
usedcars['Kilometers_Driven'] = imputed[:, 0]

# Let's see it again
fig = px.histogram(usedcars, x='Kilometers_Driven')
fig.show()

usedcars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    5847 non-null   object 
 1   Location                5847 non-null   object 
 2   Year                    5847 non-null   int64  
 3   Kilometers_Driven       5847 non-null   float64
 4   Owner_Type              5847 non-null   object 
 5   Seats                   5847 non-null   float64
 6   Price                   5847 non-null   float64
 7   Engine (CC)             5847 non-null   float64
 8   Power (bhp)             5847 non-null   float64
 9   Mileage (kmpl)          5847 non-null   float64
 10  Fuel_Type_Diesel        5847 non-null   int64  
 11  Fuel_Type_Electric      5847 non-null   int64  
 12  Fuel_Type_Petrol        5847 non-null   int64  
 13  Transmission_Automatic  5847 non-null   int64  
 14  Transmission_Manual     5847 non-null   

D) Create one more feature and add this column to the dataset.

In [ ]:
# I wanted to get a little creative wit this one and make price per km driven
usedcars['Price_per_km'] = usedcars['Price'] / usedcars['Kilometers_Driven']

# But it looks bad and I can't get it show on a log graph
fig = px.histogram(usedcars, x='Price_per_km')
fig.show()

# So here's the car age
usedcars['Car_Age'] = 2025 - usedcars['Year']
fig = px.histogram(usedcars, x='Car_Age')
fig.show()

E) Perform select, filter, rename, mutate, arrange, and summarize with group by operations on this dataset.

In [ ]:
# Select
selected = usedcars[['Name', 'Mileage (kmpl)', 'Year']]
display(selected)

,Name,Mileage (kmpl),Year
0,Hyundai Creta 1.6 CRDi SX Option,19.670000,2015
1,Honda Jazz V,17.567568,2011
2,Maruti Ertiga VDI,20.770000,2012
3,Audi A4 New 2.0 TDI Multitronic,15.200000,2013
4,Nissan Micra Diesel XV,23.080000,2013
...,...,...,...
5842,Maruti Swift VDI,28.400000,2014
5843,Hyundai Xcent 1.1 CRDi S,24.400000,2015
5844,Mahindra Xylo D4 BSIV,14.000000,2012
5845,Maruti Wagon R VXI,18.900000,2013


In [ ]:
# Filter
filtered = usedcars[(usedcars['Year'] > 2014) & (usedcars['Mileage (kmpl)'] > 20) & (usedcars['Kilometers_Driven'] < 50000)]
display(filtered)

,Name,Location,Year,Kilometers_Driven,Owner_Type,Seats,Price,Engine (CC),Power (bhp),Mileage (kmpl),Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Price_per_km,Car_Age
8,Maruti Ciaz Zeta,Kochi,2018,25692.0,First,5.0,9.95,1462.0,103.25,21.56,0,0,1,0,1,0.000387,7
15,Maruti Swift DDiS VDI,Jaipur,2017,25000.0,First,5.0,5.99,1248.0,74.00,28.40,1,0,0,0,1,0.000240,8
40,Mahindra KUV 100 mFALCON D75 K8 Dual Tone,Coimbatore,2017,22033.0,First,6.0,5.58,1198.0,77.00,25.32,1,0,0,0,1,0.000253,8
54,Renault KWID RXT,Kochi,2017,16542.0,First,5.0,3.30,799.0,53.30,25.17,0,0,1,0,1,0.000199,8
67,Hyundai Creta 1.4 E Plus Diesel,Mumbai,2018,13000.0,First,5.0,10.50,1396.0,88.70,22.10,1,0,0,0,1,0.000808,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5771,Maruti Dzire LDI,Pune,2018,27000.0,First,5.0,5.90,1248.0,73.75,28.40,1,0,0,0,1,0.000219,7
5772,Renault KWID RXL,Pune,2018,1000.0,First,5.0,3.40,799.0,53.30,25.17,0,0,1,0,1,0.003400,7
5811,Maruti Baleno Alpha,Mumbai,2017,6000.0,First,5.0,7.50,1197.0,83.10,21.40,0,0,1,0,1,0.001250,8
5816,Tata Tiago 1.2 Revotron XT,Kochi,2017,15386.0,First,5.0,5.11,1199.0,84.00,23.84,0,0,1,0,1,0.000332,8


In [ ]:
# Rename
usedcars = usedcars.rename(columns={'Name': 'Car_Model'})
display(usedcars)

,Car_Model,Location,Year,Kilometers_Driven,Owner_Type,Seats,Price,Engine (CC),Power (bhp),Mileage (kmpl),Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Price_per_km,Car_Age
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000.0,First,5.0,12.50,1582.0,126.20,19.670000,1,0,0,0,1,0.000305,10
1,Honda Jazz V,Chennai,2011,46000.0,First,5.0,4.50,1199.0,88.70,17.567568,0,0,1,0,1,0.000098,14
2,Maruti Ertiga VDI,Chennai,2012,87000.0,First,7.0,6.00,1248.0,88.76,20.770000,1,0,0,0,1,0.000069,13
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670.0,Second,5.0,17.74,1968.0,140.80,15.200000,1,0,0,1,0,0.000436,12
4,Nissan Micra Diesel XV,Jaipur,2013,86999.0,First,5.0,3.50,1461.0,63.10,23.080000,1,0,0,0,1,0.000040,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,Maruti Swift VDI,Delhi,2014,27365.0,First,5.0,4.75,1248.0,74.00,28.400000,1,0,0,0,1,0.000174,11
5843,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000.0,First,5.0,4.00,1120.0,71.00,24.400000,1,0,0,0,1,0.000040,10
5844,Mahindra Xylo D4 BSIV,Jaipur,2012,55000.0,Second,8.0,2.90,2498.0,112.00,14.000000,1,0,0,0,1,0.000053,13
5845,Maruti Wagon R VXI,Kolkata,2013,46000.0,First,5.0,2.65,998.0,67.10,18.900000,0,0,1,0,1,0.000058,12


In [ ]:
# Mutate, we already did this above
usedcars['Mileage_per_year'] = usedcars['Mileage (kmpl)'] / usedcars['Car_Age']
display(usedcars[['Mileage (kmpl)', 'Car_Age', 'Mileage_per_year']])

,Mileage (kmpl),Car_Age,Mileage_per_year
3630,10.00,27,0.370370
3039,17.30,27,0.640741
5558,17.30,27,0.640741
1185,17.30,26,0.665385
1791,13.00,26,0.500000
...,...,...,...
2949,12.63,6,2.105000
3445,13.68,6,2.280000
3220,17.90,6,2.983333
5628,25.17,6,4.195000


In [ ]:
# Arrange
usedcars = usedcars.sort_values(by='Year')
display(usedcars)

,Car_Model,Location,Year,Kilometers_Driven,Owner_Type,Seats,Price,Engine (CC),Power (bhp),Mileage (kmpl),Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Price_per_km,Car_Age,Mileage_per_year
3630,Mercedes-Benz E-Class 250 D W 210,Mumbai,1998,55300.0,First,5.0,3.90,1796.0,157.7,10.00,1,0,0,1,0,0.000071,27,0.370370
3039,Maruti Zen LXI,Jaipur,1998,95150.0,Third,5.0,0.45,993.0,60.0,17.30,0,0,1,0,1,0.000005,27,0.640741
5558,Maruti Zen LX,Jaipur,1998,95150.0,Third,5.0,0.53,993.0,60.0,17.30,0,0,1,0,1,0.000006,27,0.640741
1185,Maruti Zen VX,Jaipur,1999,70000.0,Second,5.0,0.77,993.0,60.0,17.30,0,0,1,0,1,0.000011,26,0.665385
1791,Honda City 1.3 EXI,Pune,1999,140000.0,First,5.0,0.90,1343.0,90.0,13.00,0,0,1,0,1,0.000006,26,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2949,Land Rover Discovery Sport TD4 HSE,Coimbatore,2019,25899.0,First,5.0,55.70,2179.0,147.5,12.63,1,0,0,1,0,0.002151,6,2.105000
3445,Toyota Innova Crysta 2.4 VX MT,Coimbatore,2019,33598.0,First,7.0,22.20,2393.0,147.8,13.68,1,0,0,0,1,0.000661,6,2.280000
3220,Mercedes-Benz CLA 200 CDI Sport,Kochi,2019,29836.0,First,5.0,25.75,2143.0,136.0,17.90,1,0,0,1,0,0.000863,6,2.983333
5628,Renault KWID RXL,Kochi,2019,7858.0,First,5.0,3.80,799.0,53.3,25.17,0,0,1,0,1,0.000484,6,4.195000


In [ ]:
# Summarize with groupby
usedcars.groupby('Year').agg({'Price': ['mean', 'min', 'max']})

Price              
           mean   min     max
Year                         
1998   1.626667  0.45    3.90
1999   0.835000  0.77    0.90
2000   1.175000  0.55    1.95
2001   0.920000  0.70    1.45
2002   1.321667  0.51    3.65
2003   2.548750  0.50   16.11
2004   1.990000  0.45    7.10
2005   2.262955  0.45   10.00
2006   3.596377  0.69   59.00
2007   3.427830  0.75   29.00
2008   4.051227  0.60   40.88
2009   5.246269  0.80   36.00
2010   5.477970  0.50   42.91
2011   6.813341  0.44  120.00
2012   7.460699  0.53   70.00
2013   8.835800  1.20   75.00
2014   9.742462  1.00   90.00
2015  11.316962  1.50  100.00
2016  11.776798  1.80   79.00
2017  13.831735  1.60  160.00
2018  15.884558  2.90   93.67
2019  19.603564  2.02   97.07